# 1. Install, Imports, Settings

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import spacy
from spacy.lang.es.examples import sentences 
#!python -m spacy download es_core_news_md
nlp = spacy.load("es_core_news_md")
from collections import Counter
import string, math, json

In [2]:
plt.rcParams["figure.figsize"] = [10, 6]
%config InlineBackend.figure_format = 'retina'

punct = string.punctuation +'”“'

# 2. Loading data

In [3]:
grupos = {
    "Grupo Armado Organizado" : ["GAO"],
    "Grupo Armado Ilegal" : ["GAI"],
    "Grupo Armado Organizado Residual" : ["GAOR", "GAO-R", "grupo armado organizado - residual" ], 
    "Ejército de Liberación Nacional" : ["ELN"],
    "Autodefensas Gaitanistas de Colombia" : ["AGC"],
    "Fuerzas Armadas Revolucionarias - Ejército del Pueblo" : ["FARC", 
                                         "Fuerzas Armadas Revolucionarias de Colombia",
                                         "Fuerzas Armadas Revolucionarias Ejército del Pueblo",
                                         "FARCEP", "FARC-EP", "FARC EP"
                                        ],
    "Grupo delincuencial organizado" : ["GDO", ],
    "Jalisco Nueva Generación": [],
    "Bandas Criminales" : ["BACRIM", "Bandas Emergentes"],
    "Autodefensas Unidas de Colombia" : ["AUC", ],
    "Grupos posdesmovilización": [],
    "Ejército de Liberación Popular": ["EPL"],
    "Clan del Golfo" : ["AGC"],
}

children = ["NNAJ", "niños", "niñas", "adolescentes", "jóvenes", "NNA"]

narco_grupos  = ["Cartel del Amazonas",  
        "Cartel de Los Nevados",
        "Cartel de Bogotá",
        "Cartel de Buga",
        "Cartel de Medellín",
        "Oficina de Envigado",
        "Cartel de Cali",
        "Cartel del Caquetá",
        "Cartel del Norte del Valle",
        "Los Rastrojos",
        "Cartel de la Costa Atlántica",
        "Cartel de La Guajira",
        "Cartel del Cauca",
        "Los Caparrapos",
        "Los Paisas",
        "Águilas Negras",
        "Los Puntilleros",
        "Ejército Revolucionario Popular Antisubversivo de Colombia",
        "Los Pelusos",
        "GDO Los Pachenca",
        "Clan Gnecco Cerchar",
        "Clan Bustamante", 
        "Activo Primer Frente",
        "Segunda Marquetalia", 
        "Los Zetas", 
        "Cartel de Sinaloa", 
        "Oficina de Envigado", 
        "La Terraza",
        "Robledo",
        "Pachelly",
        "La Sierra", 
        "Caicedo", 
        "Los Triana",
        "Clan del Oriente", 
        "La Unión",
        "Clan del Norte",
        "Los Costeños", 
        "Los Pachencha", 
        "La Cordillera",
        "La Gran Alianza",
        "Banda Local",
        "Norte del Valle",
        "Los Chuckys",
        "Los Chacales",
        "La Constru",
        "Los Caqueteños"]

POI = {
    "Evaristo Porras": "Papa Doc",
    "Luis Agustín Caicedo Velandia" : "Don Lucho", 
    "Víctor Manuel Mejía Múnera": "Pablo Arauca",
    "Miguel Ángel Mejía Múnera": "El Mellizo",
    "Ramón Quintero":"",
    "Pablo Escobar" : "El patrón",
    "Gonzalo Rodríguez Gacha": "",
    "Diego Fernando Murillo": "",
    "Gilberto Rodríguez Orejuela": "El Ajedrecista",
    "Miguel Rodríguez Orejuela": "",
    "Leonidas Vargas": "El Rey de Caquetá",
    "Luis Enrique Calle Serna": "",
    "Wilber Varela": "",
    "Alberto Orlandez Gamboa": "", 
    "Hermágoras González": "",
    "Nicolás Rodríguez Bautista" : " Gabino ", 
    "Iván Márquez": "",
    "Gentil Duarte": "",
    "Jorge Gnecco Cerchar": "", 
    "Jesús María Aguirre" :"Chucho Mercancía",
    "Megateo": "",  
    "Vicente Castaño": "", 
    "Emiliano Alcides Osorio": " Caín ",
    "Edgar Bustamante" : "El Yuca",
    "Manuel Marulanda Vélez" :" Tirofijo ",     
    "Jorge Briceño Suarez Mono": " Jojoy ",
    "Dairo Antonio Úsuga David": "",
    "Salvatore Mancuso": "",
    "Rodrigo Tovar": "",
    "Carlos Castaño Gil": "",
    "Justo Pastor Perafán": "Don Pepe"
}

loc = pd.read_csv("resources/Departamentos_y_municipios_de_Colombia.csv")
loc.head()

,REGION,CÓDIGO DANE DEL DEPARTAMENTO,DEPARTAMENTO,CÓDIGO DANE DEL MUNICIPIO,MUNICIPIO
0,Región Eje Cafetero - Antioquia,5,Antioquia,5.001,Medellín
1,Región Eje Cafetero - Antioquia,5,Antioquia,5.002,Abejorral
2,Región Eje Cafetero - Antioquia,5,Antioquia,5.004,Abriaquí
3,Región Eje Cafetero - Antioquia,5,Antioquia,5.021,Alejandría
4,Región Eje Cafetero - Antioquia,5,Antioquia,5.030,Amagá


In [7]:
dfnnaj = pd.read_csv("data/alertas_actors_NNAJ.csv", sep="|")
df = pd.read_csv("data/alertas_actors.csv", sep="|")
#dfnnaj = dfnnaj.drop(['Unnamed: 0'], axis=1)
print("Total Number of documents:", len(df))
print("Total Number of documents NNAJ:", len(dfnnaj)) 

Total Number of documents: 1752
Total Number of documents NNAJ: 1140


In [8]:
df.head()

,Unnamed: 0,Filename,Subtype,Type,Year,Path,Departamento,Actors
0,0,"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Polic...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
1,1,AT N° 004-18 NAR-Tumaco.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
2,2,AT N° 005-18 COR-Tierralta.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
3,3,AT N° 006-18 ARA-Saravena.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
4,4,"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y ...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Grupo Armado Organizado', 'Grupo Armado Orga..."


In [9]:
dfnnaj.tail(5)

,Filename,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,Actors
1135,IS N° 004-2022 a AT 048-20 PUT-Puerto Leguíza...,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 004-20...,Putumayo,"['Instituto Colombiano de Bienestar Familiar',...","['Grupo Armado Ilegal', 'La Constru', 'Doctora..."
1136,IS N° 005-2022 a AT N° 041-19 CHO-Alto Baudó y...,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 005-20...,Chocó,"['Frente de Guerra Occidental', 'Ejército de L...","['Ejército de Liberación Nacional', 'Doctora M..."
1137,IS N° 006-2022 a AT N° 004-2021.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 006-20...,Norte de Santander,"['La Constru', 'La Constru', 'Servicio Naciona...","['La Constru', 'La Constru', 'La Constru', 'La..."
1138,IS N° 011-2022 a AT N° 031-2020 MET-San Martín...,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 011-20...,Meta,"['Autodefensas Gaitanistas de Colombia', 'Band...","['Autodefensas Gaitanistas de Colombia', 'Auto..."
1139,IS N° 012-2022 a AT N° 042 de 2020 BOY-El Cocu...,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 012-20...,Boyacá,"['Grupo Armado Ilegal', 'La Sierra', 'La Sierr...","['Grupo Armado Ilegal', 'La Sierra', 'La Sierr..."


# 3. Cleaning Duty

In [5]:
arts = ['la', 'los', 'el', 'las', 'una', 'un']
def replace_accents(w):
    return w.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')

arts = ['la', 'los', 'el', 'las', 'una', 'un']
def replace_articles(w):
    w = [x for x in w.split() if x not in arts]
    return ' '.join(w)

def clean_segment(s):
    return replace_articles(replace_accents(s.lower()))
    
def clean_actors(entities):
    entities = entities.strip("']['").split("', '")
   
    if len(entities) == 0:
        return []
    
    #grupos
    for key, value in grupos.items():
        for v in value:
            entities = [key if clean_segment(e) == clean_segment(v) else e for e in entities]
           
    #narco grupos
    for v in narco_grupos:
        entities = [v if clean_segment(e) == clean_segment(v) else e for e in entities]
    return entities

df["Actors"] = df["Actors"].apply(clean_actors)
dfnnaj["Actors_NNAJ"] = dfnnaj["Actors_NNAJ"].apply(clean_actors)

In [6]:
len(df), len(dfnnaj)

(1752, 1140)

In [10]:
df.to_csv("data/alertas_actors.csv", sep="|")
df = df.set_index('Filename')
#remove duplicates
df = df[~df.index.duplicated(keep='first')]
df = df
df.head()

,Unnamed: 0,Subtype,Type,Year,Path,Departamento,Actors
Filename,,,,,,,
"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Policarpa y Roberto Payán.pdf",0,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
AT N° 004-18 NAR-Tumaco.pdf,1,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
AT N° 005-18 COR-Tierralta.pdf,2,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
AT N° 006-18 ARA-Saravena.pdf,3,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y Vista Hermosa.pdf",4,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Grupo Armado Organizado', 'Grupo Armado Orga..."


In [11]:
dfnnaj.to_csv("data/alertas_actors_NNAJ.csv", sep="|")
dfnnaj = dfnnaj.set_index('Filename')
dfnnaj.tail()

,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,Actors
Filename,,,,,,,
IS N° 004-2022 a AT 048-20 PUT-Puerto Leguízamo.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 004-20...,Putumayo,"['Instituto Colombiano de Bienestar Familiar',...","['Grupo Armado Ilegal', 'La Constru', 'Doctora..."
IS N° 005-2022 a AT N° 041-19 CHO-Alto Baudó y AT N° 030-20 CHO-Bajo Baudó.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 005-20...,Chocó,"['Frente de Guerra Occidental', 'Ejército de L...","['Ejército de Liberación Nacional', 'Doctora M..."
IS N° 006-2022 a AT N° 004-2021.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 006-20...,Norte de Santander,"['La Constru', 'La Constru', 'Servicio Naciona...","['La Constru', 'La Constru', 'La Constru', 'La..."
IS N° 011-2022 a AT N° 031-2020 MET-San Martín.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 011-20...,Meta,"['Autodefensas Gaitanistas de Colombia', 'Band...","['Autodefensas Gaitanistas de Colombia', 'Auto..."
"IS N° 012-2022 a AT N° 042 de 2020 BOY-El Cocuy, Güicán de la Sierra, Panqueba, El Espino, Chiscas, Chita, Jericó, Socha y Socotá.pdf",Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 012-20...,Boyacá,"['Grupo Armado Ilegal', 'La Sierra', 'La Sierr...","['Grupo Armado Ilegal', 'La Sierra', 'La Sierr..."


In [13]:
#df = df.to_frame()
df.head()

,Unnamed: 0,Subtype,Type,Year,Path,Departamento,Actors
Filename,,,,,,,
"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Policarpa y Roberto Payán.pdf",0,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
AT N° 004-18 NAR-Tumaco.pdf,1,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
AT N° 005-18 COR-Tierralta.pdf,2,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
AT N° 006-18 ARA-Saravena.pdf,3,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y Vista Hermosa.pdf",4,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Grupo Armado Organizado', 'Grupo Armado Orga..."


In [14]:
#merging DFs
dftotal = pd.concat([dfnnaj, df.reindex(dfnnaj.index)], axis=1)
len(dftotal)

1140

In [15]:
dftotal[dftotal['Actors_NNAJ'].isnull()]

,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,Actors,Unnamed: 0,Subtype,Type,Year,Path,Departamento,Actors
Filename,,,,,,,,,,,,,,


In [16]:
dftotal = dftotal.reset_index()
dftotal.head()

,Filename,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,Actors,Unnamed: 0,Subtype,Type,Year,Path,Departamento,Actors
0,"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Polic...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['RECOMENDACIONES A', 'Comisión lntersectorial']","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",0,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
1,AT N° 004-18 NAR-Tumaco.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Fuerzas Armadas Revol...","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",1,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
2,AT N° 005-18 COR-Tierralta.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Instituto Colombiano ...","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",2,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
3,AT N° 006-18 ARA-Saravena.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Derecho Internacional Humanitario', 'Policía']","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",3,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
4,"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y ...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Nuev', 'Partido FAR', 'Defensoria del Pueblo...","['Grupo Armado Organizado', 'Grupo Armado Orga...",4,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Grupo Armado Organizado', 'Grupo Armado Orga..."


In [17]:
dftotal.to_csv("data/alertas_actors_NNAJ.csv", sep="|", index=False)

# 4. Functions

In [18]:
df = pd.read_csv("data/alertas_actors_NNAJ.csv", sep="|")
df.head()

,Filename,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,Actors,Unnamed: 0,Subtype.1,Type.1,Year.1,Path.1,Departamento.1,Actors.1
0,"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Polic...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['RECOMENDACIONES A', 'Comisión lntersectorial']","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",0,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
1,AT N° 004-18 NAR-Tumaco.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Fuerzas Armadas Revol...","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",1,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
2,AT N° 005-18 COR-Tierralta.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Instituto Colombiano ...","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",2,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
3,AT N° 006-18 ARA-Saravena.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Derecho Internacional Humanitario', 'Policía']","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",3,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
4,"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y ...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Nuev', 'Partido FAR', 'Defensoria del Pueblo...","['Grupo Armado Organizado', 'Grupo Armado Orga...",4,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Grupo Armado Organizado', 'Grupo Armado Orga..."


In [19]:
space_articles = []
red_flags = ['Telefax', 'procesoSubproceso Prevención y Protección Código','SAT', 'Alerta Temprana', 
             'Derecho Internacional Humanitario', 'SAT Código DPR', 
             'procesoSubproceso Prevención y Protección Código PPP02F10 ALERTA', 'SEGUIMIENTO', 
             'PARA', 'Frente', 'Nota de Seguimiento', 'PP', 'AT', 'LIBERTAD',  'Alertas Tempranas', 
             'P01 F07 Formato Nota de Seguimiento Versión 03 Vigente', 
             'Prevención y Protección Código PP', 'Bloque C', 'Comisión', 'POBLACION','Constitución Política', 
             'procesoSubproceso Prevención y Protección Código PP', 'Ejército', 'Sin', 'INFORME', 'MUSE', 
             'BIENES INDISPENSABLES PARA', 'Gobernación', 'A NO SER', 'cia', 'Alerta', 'RECOMENDACIONES A', 
            'NNA', 'Adolescentes', 'ADOLESCENTES', 'SAT INFORME DE RIESGO No']

arts = ['la', 'los', 'el', 'las']

In [20]:
def remove_flags(entities):
     return [e for e in entities if e not in red_flags]

def remove_articles(entities):
    new = []
    for i in range(0, len(entities) - 1):
            new.append(entities[i])
            for a in arts:
                if entities[i].lower().startswith(a + " "):
                    new[len(new)-1] = entities[i][len(a):]

    return new
                     
def fix_terms(entities):
    new = []
    for i in range(0, len(entities)):
        new.append(entities[i])
        # fix ELN
        if entities[i].startswith("ELN "):
                   new[len(new) - 1] =  "Ejército de Liberación Nacional"
        #fix Las FARC           
        elif entities[i].startswith("FARC") or entities[i].startswith("lasFARC") or entities[i].startswith("LasFARC"):
                   new[len(new) - 1] = "Fuerzas Armadas Revolucionarias - Ejército del Pueblo" 
                
        elif entities[i].startswith("Fuerzas Armadas Revolucionarias"):
                   new[len(new) - 1] = "Fuerzas Armadas Revolucionarias - Ejército del Pueblo" 
                
        elif entities[i].lower().startswith("urabe"):
                   new[len(new) - 1] = "Urabeños"     
                
        elif entities[i].startswith("UNP"):
                   new[len(new) - 1] = "Unidad Nacional de Protección"    
        
        elif entities[i].startswith("Acción Social"):
                   new[len(new) - 1] = "Agencia Presidencial para la Acción Social"
                
        elif entities[i].startswith("Al Instituto Colombiano de Bienestar Familiar") or entities[i].startswith("ICBF"):
                   new[len(new) - 1] = "Instituto Colombiano de Bienestar Familiar"
                
        elif entities[i].lower().startswith("armada"):
                   new[len(new) - 1] = "Armada Nacional"
                
        elif entities[i].lower().startswith("defensor"):
                   new[len(new) - 1] = "Defensoría del Pueblo"
                
    return new

def clean_list_actors(df, column):
    actors = df[column].sum()
    if type(actors) == str:
        actors = actors.strip("']['").split("', '")
    
    #remove articles
    actors = remove_articles(actors)
                     
    #fix some terms
    actors = fix_terms(actors)                               
                
    #remove red flags
    actors = remove_flags(actors)
    
    #remove blank spaces
    actors = [a.strip() for a in actors]
    
    return actors
    
def most_mentioned_actors(actors):           
    data = { 
             'actors': list(Counter(actors).keys()), 
             'frequency': list(Counter(actors).values())
           }

    counterdf = pd.DataFrame(data)
    counterdf = counterdf.groupby('actors').agg({'frequency' : 'sum'}).sort_values('frequency', ascending = False)
    return counterdf

# 4. Most Mentioned Actors - Top 20 in Documents that mention NNAJ

#### Los articulos se omiten por cuestiones de estandarizacion de los datos, pero por ejemplo Rastrojos se refiere a Los Rastrojos que es el nombre original, as'i pasa con Los Paisas, La Cordillera, La Constru, etc. 

In [21]:
dfa = most_mentioned_actors(clean_list_actors(df, 'Actors'))
dfa[:20]

,frequency
actors,
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,11692
Ejército de Liberación Nacional,6024
Autodefensas Unidas de Colombia,5175
Fuerza Pública,4560
Águilas Negras,3347
Rastrojos,2587
Policía Nacional,2484
Autodefensas Gaitanistas de Colombia,2083
Grupo Armado Ilegal,1685


# 4.1 Most Mentioned Actors - Top 20 in the Same Sentence that mention  NNAJ - More Context

In [24]:
dfb = most_mentioned_actors(clean_list_actors(df, 'Actors_NNAJ'))
dfb[:20]

,frequency
actors,
Instituto Colombiano de Bienestar Familiar,628
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,555
Ejército de Liberación Nacional,299
Autodefensas Unidas de Colombia,256
Fuerza Pública,169
Águilas Negras,162
Rastrojos,135
Urabeños,109
Policía Nacional,98


In [27]:
df.head()

,Filename,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,Actors,Unnamed: 0,Subtype.1,Type.1,Year.1,Path.1,Departamento.1,Actors.1
0,"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Polic...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['RECOMENDACIONES A', 'Comisión lntersectorial']","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",0,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
1,AT N° 004-18 NAR-Tumaco.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Fuerzas Armadas Revol...","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",1,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
2,AT N° 005-18 COR-Tierralta.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Instituto Colombiano ...","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",2,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
3,AT N° 006-18 ARA-Saravena.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Derecho Internacional Humanitario', 'Policía']","['Grupo Armado Ilegal', 'Grupo Armado Ilegal',...",3,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
4,"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y ...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Nuev', 'Partido FAR', 'Defensoria del Pueblo...","['Grupo Armado Organizado', 'Grupo Armado Orga...",4,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Grupo Armado Organizado', 'Grupo Armado Orga..."


# 5. Statistical Distribution of Actors

In [133]:
dfa[dfa.frequency > 100].describe()

,frequency
count,108.000000
mean,637.009259
std,1452.174049
min,102.000000
25%,129.500000
50%,173.500000
75%,459.500000
max,11692.000000


# 6. Top 10 actors in each period

In [134]:
df1 = df[df.Year < 2006]
df2 = df[(df.Year > 2005) & (df.Year < 2012)]
df3 = df[(df.Year > 2011) & (df.Year < 2017)]
df4 = df[df.Year > 2016]

# 6.1 2001 - 2005 (Conflicto, 2005 Justicia y Paz)

In [135]:
df1 = most_mentioned_actors(clean_list_actors(df1, 'Actors'))[:10]
df1

,frequency
actors,
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,891
Autodefensas Unidas de Colombia,888
Ejército de Liberación Nacional,185
Fuerza Pública,174
Autodefensas,152
Red de Solidaridad Social,96
Armada Nacional,86
Cordillera,73
Policía,49


In [136]:
df1[df1.frequency > 100].describe()

,frequency
count,5.000000
mean,458.000000
std,394.084382
min,152.000000
25%,174.000000
50%,185.000000
75%,888.000000
max,891.000000


# 6.2 2006- 2011 (Conflicto)

In [137]:
df2 = most_mentioned_actors(clean_list_actors(df2, 'Actors'))[:10]
df2

,frequency
actors,
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,3253
Autodefensas Unidas de Colombia,1849
Águilas Negras,1691
Fuerza Pública,1355
Rastrojos,992
Ejército de Liberación Nacional,903
Agencia Presidencial para la Acción Social,773
Grupo Armado Ilegal,435
Policía Nacional,422


In [138]:
df2[df2.frequency > 100].describe()

,frequency
count,10.000
mean,1206.900
std,885.147
min,396.000
25%,519.500
50%,947.500
75%,1607.000
max,3253.000


# 6.3 2012 - 2016 (Negociaciones de Paz, Acuerdo de La Habana)

In [139]:
df3 = most_mentioned_actors(clean_list_actors(df3, 'Actors'))[:10]
df3

,frequency
actors,
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,3179
Ejército de Liberación Nacional,1246
Fuerza Pública,1245
Rastrojos,1224
Autodefensas Unidas de Colombia,1066
Urabeños,1055
Águilas Negras,1049
Autodefensas Gaitanistas de Colombia,725
Policía Nacional,666


In [140]:
df3[df3.frequency > 100].describe()

,frequency
count,10.000000
mean,1201.200000
std,738.403353
min,557.000000
25%,806.000000
50%,1060.500000
75%,1239.750000
max,3179.000000


# 6.4 2017 - 2022 (Post- Acuerdo)

In [141]:
df4 = most_mentioned_actors(clean_list_actors(df4, 'Actors'))[:10]
df4

,frequency
actors,
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,4369
Ejército de Liberación Nacional,3690
Fuerza Pública,1786
Autodefensas Unidas de Colombia,1372
Policía Nacional,1367
Autodefensas Gaitanistas de Colombia,1164
Constru,969
Unidad Nacional de Protección,947
Grupo Armado Ilegal,688


In [142]:
df4[df4.frequency > 100].describe()

,frequency
count,10.000000
mean,1695.900000
std,1287.410014
min,607.000000
25%,952.500000
50%,1265.500000
75%,1682.500000
max,4369.000000


# 6. Export HTML

In [144]:
!jupyter nbconvert --to html 14_Mapping_Actores_del_Conflicto_NNAJ.ipynb

[NbConvertApp] Converting notebook 14_Mapping_Actores_del_Conflicto_NNAJ.ipynb to html
[NbConvertApp] Writing 690663 bytes to 14_Mapping_Actores_del_Conflicto_NNAJ.html
